In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def ann_regression_lags_only(
    series,
    district_name,
    max_lags=12,
    differencing=True,
    epochs=100,
    batch_size=32,
    hidden_layers=[32, 16],
    activation='relu',
    verbose=0
):
    """
    ANN regression for time series forecasting with lag features.
    Returns (None, None) on error to maintain pipeline flow.
    """
    try:
        # 0. Initial checks
        if len(series) < max_lags + 24:
            raise ValueError(f"Series too short ({len(series)} points) for {max_lags} lags")

        # 1. Directory setup
        os.makedirs('ANN_Forecasts', exist_ok=True)
        
        # 2. Stationarity handling
        original_series = series.copy()
        d = 0
        if differencing:
            adf_result = adfuller(series.dropna())
            if adf_result[1] > 0.05:
                d = 1
                series = series.diff().dropna()
                if len(series) < 10:
                    raise ValueError("Insufficient data after differencing")

        # 3. Create lag features
        df = pd.DataFrame({'y': series})
        for lag in range(1, max_lags + 1):
            df[f'lag_{lag}'] = df['y'].shift(lag)
        df = df.dropna()

        # 4. Temporal split with index preservation
        train_size = int(len(df) * 0.8)
        if train_size < 10 or (len(df) - train_size) < 2:
            raise ValueError("Insufficient train/test split")
            
        train = df.iloc[:train_size]
        test = df.iloc[train_size:]
        test_series = test['y']  # Preserve for index

        # 5. Prepare numpy arrays
        X_train = train.drop(columns=['y']).values
        y_train = train['y'].values.reshape(-1, 1)
        X_test = test.drop(columns=['y']).values
        y_test_np = test['y'].values.reshape(-1, 1)

        # 6. Scaling
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_test_scaled = scaler_y.transform(y_test_np)

        # 7. Build ANN model
        model = Sequential()
        model.add(Dense(hidden_layers[0], activation=activation, input_dim=max_lags))
        for units in hidden_layers[1:]:
            model.add(Dense(units, activation=activation))
        model.add(Dense(1))  # Output layer
        model.compile(optimizer='adam', loss='mse')

        # 8. Train with validation
        history = model.fit(
            X_train_scaled, 
            y_train_scaled,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_split=0.2,
            shuffle=False
        )

        # 9. Predict and inverse scaling
        y_pred_scaled = model.predict(X_test_scaled, verbose=verbose)
        y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()

        # 10. Inverse differencing
        if d == 1:
            last_train_value = original_series.iloc[train.index[0] - 1]
            y_pred = np.cumsum(y_pred) + last_train_value
            y_test_values = original_series.iloc[test.index].values
        else:
            y_test_values = test_series.values

        # 11. Create output (preserve original indices)
        forecast_df = pd.DataFrame({
            'district': district_name,
            'date': test_series.index,
            'actual': y_test_values,
            'forecast': y_pred
        })

        # 12. Calculate metrics
        rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))

        # 13. Save results
        forecast_df.to_csv(
            os.path.join('ANN_Forecasts', f'{district_name}_forecast.csv'), 
            index=False
        )

        # 14. Plotting
        plt.figure(figsize=(12, 6))
        plt.plot(original_series.index, original_series, label='Original')
        plt.plot(forecast_df['date'], forecast_df['forecast'], label='ANN Forecast')
        plt.title(f'{district_name} ANN Forecast (RMSE: {rmse:.2f})')
        plt.legend()
        plt.savefig(os.path.join('ANN_Forecasts', f'{district_name}_forecast_plot.png'))
        plt.close()

        return {
            'district': district_name,
            'rmse': rmse,
            'epochs': epochs,
            'batch_size': batch_size,
            'training_loss': history.history['loss'],
            'validation_loss': history.history['val_loss']
        }, forecast_df

    except Exception as e:
        print(f"⚠️ Skipping {district_name}: {str(e)}")
        return None, None

In [4]:
districts = data['district'].unique()
rmse_values = []

# Usage Example


def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I40"].asfreq('MS')
        
        results, forecast = ann_regression_lags_only(
            series=ts,
            district_name=district,
            max_lags=6,
            epochs=200,
            batch_size=4,
            hidden_layers=[32, 16],
            activation='relu',
            verbose=1
        )

        
    
        # Show results
        print("=== Metrics ===")
        print(results)
        print("\n=== Forecast Data ===")
        print(forecast)
    
    return results
run_for_each_district()

Epoch 1/200
7/7 [==============================] - 0s 15ms/step - loss: 0.3595 - val_loss: 0.1387
Epoch 2/200
7/7 [==============================] - 0s 7ms/step - loss: 0.2603 - val_loss: 0.0924
Epoch 3/200
7/7 [==============================] - 0s 4ms/step - loss: 0.1968 - val_loss: 0.0693
Epoch 4/200
7/7 [==============================] - 0s 6ms/step - loss: 0.1607 - val_loss: 0.0549
Epoch 5/200
7/7 [==============================] - 0s 4ms/step - loss: 0.1389 - val_loss: 0.0457
Epoch 6/200
7/7 [==============================] - 0s 6ms/step - loss: 0.1188 - val_loss: 0.0391
Epoch 7/200
7/7 [==============================] - 0s 4ms/step - loss: 0.1001 - val_loss: 0.0349
Epoch 8/200
7/7 [==============================] - 0s 4ms/step - loss: 0.0833 - val_loss: 0.0339
Epoch 9/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0691 - val_loss: 0.0369
Epoch 10/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0583 - val_loss: 0.0441
Epoch 11/200
7/7 [==========